In [ ]:
'''
普通Groupby并行迭代器
'''

In [ ]:
from joblib import Parallel, delayed
import multiprocessing

In [ ]:
def applyParallel(dfGrouped, func):
    retLst = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(group) for name, group in dfGrouped)
    return pd.concat(retLst,axis=0)

return_MKT = applyParallel(price_df.groupby('code'), merge_price_MKT).rename(columns={'0_x':'return','0_y':'MKT','code_x':'code','code_y':'tmp'}).reset_index()[['code','date','return','MKT']]
return_MKT = return_MKT[return_MKT['code']!='MKT'][['code','date','return','MKT']]
t2 = datetime.datetime.now()

In [ ]:
'''
滚动窗口（多列数据）
'''

In [ ]:
# 首先生成一个用于循环计算的迭代器
DfList = []
dateList = AllDataReturn.index.tolist()
for dateCnt in range(len(dateList)):
    date = dateList[dateCnt]
    AllDataReturnSub = AllDataReturn[AllDataReturn.index<=date].copy()
    AllDataReturnSub = AllDataReturnSub.iloc[-win:,:].copy()
    DfList.append(AllDataReturnSub)

t1 = datetime.datetime.now()

# 对迭代器中每个元素的处理函数，长度不及滚动窗口则返回np.nan，否则使用计算函数
def ComputeCorr(df):
    corrList = []
    for col in [i for i in df.columns if i != '沪深300']:
        corrList.append(np.corrcoef(df['沪深300'],df[col])[0,1])
        corr = np.mean(corrList)
    return corr
def MultiTask(df):
    df = df.copy()
    if len(df)<252:
        corr = np.nan
    else:
        corr = ComputeCorr(df)
    return corr

saveCorrList = Parallel(n_jobs=mp.cpu_count())(delayed(MultiTask)(group) for group in DfList)

RollingCorr = pd.DataFrame(saveCorrList,columns=['RollingCorr'],index = dateList)
print(datetime.datetime.now()-t1)